In [87]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import sys
import math
import os
from os import walk, remove
%matplotlib inline

files = next(walk('./input'), (None, None, []))[2]  # [] if no file
outputFiles = next(walk('./output'), (None, None, []))[2]  # [] if no file
for outputFile in outputFiles:
    remove('./output/' + outputFile)


def filterFunction(cnt, image):
    x, y, w, h = cv2.boundingRect(cnt)
    if h > (image.shape[0] / 2) and w < (image.shape[1] / 5) and x > (3 * image.shape[1] / 4):
        return True
    return False

def filterFunction2(cnt, image):
    x, y, w, h = cv2.boundingRect(cnt)
    if h > (image.shape[0] / 30):
        return True
    return False

for path in files:
    image = cv2.imread(f'./input/{path}')
    # cv2.imwrite(f"./output/{path.split('.')[0]}.png", image)  
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    im_gray = cv2.imread(f'./input/{path}', cv2.IMREAD_GRAYSCALE)
    image_enhanced = cv2.equalizeHist(im_gray)
    thresh = 200
    im_bw = cv2.threshold(image_enhanced, thresh, 255, cv2.THRESH_BINARY)[1]
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 3))
    dilate = cv2.dilate(im_bw, kernel, iterations=25)
    # cv2.imwrite(f"./output/{path.split('.')[0]}_dilatedY.png", dilate) 

    cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    cnts = filter(lambda item: filterFunction(item, im_bw), cnts)
    cnts = sorted(cnts, key=lambda item1: item1[0][0][0], reverse=True)
    if(len(cnts) > 0):
        x, y, w, h = cv2.boundingRect(cnts[0])
        # cv2.imwrite(f"./output/{path.split('.')[0]}_cropped.png", image[:, 0:x - 10])
        image = image[:, 0:x - 10] #image without friend list
        im_gray = im_gray[:, 0:x - 10]
        image_enhanced = cv2.equalizeHist(im_gray)
    
    thresh = 250
    im_bw = cv2.threshold(image_enhanced, thresh, 255, cv2.THRESH_BINARY)[1]

    # cv2.imwrite(f"./output/{path.split('.')[0]}_black.png", im_bw) 
    # image ja no te la part dels amics, ara treurem la part de dalt i baix

    # Dilate to combine adjacent text contours
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (20, 1))
    dilate = cv2.dilate(im_bw, kernel, iterations=100)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 3))
    dilate = cv2.dilate(dilate, kernel, iterations=1)
    # cv2.imwrite(f"./output/{path.split('.')[0]}_dilatedH.png", dilate)  

    cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    cnts = filter(lambda item: filterFunction2(item, im_bw), cnts)
    cnts = sorted(cnts, key=lambda item1: item1[0][0][1])
    # for cnt in cnts:
    #     cv2.drawContours(image, [cnt], 0, (255,255,0), 3)
    # cv2.imwrite(f"./output/{path.split('.')[0]}_cnts.png", image)  

    top = None
    bottom = None
    if(len(cnts) > 0):
        foundTop = False
        foundBottom = False
        cnts2 = cnts[:]
        victoryImg = None
        while not foundTop:
            x, y, w, h = cv2.boundingRect(cnts2[0])
            x2, y2, w2, h2 = cv2.boundingRect(cnts2[1])
            if(y2 - (y + h) > image.shape[0] / 20):
                victoryImg = image[0:y + h, :]
                # cv2.imwrite(f"./output/{path.split('.')[0]}_victory.png", victoryImg)
                top = y2
                foundTop = True
            elif(y > image.shape[0] / 10): 
                top = y
                foundTop = True
            else:
                cnts2.pop(0)
            if(len(cnts2) < 2):
                top = 0
                foundTop = True

        cnts2 = cnts[:]
        while not foundBottom:
            x, y, w, h = cv2.boundingRect(cnts2[-1])
            x2, y2, w2, h2 = cv2.boundingRect(cnts2[-2])
            if(y - (y2 + h2) > image.shape[0] / 20):
                bottom = y2 + h2
                foundBottom = True
            elif(y > image.shape[0] / 2):
                bottom = y + h
                foundBottom = True
            else:
                cnts2.pop(-1)
            if(len(cnts2) < 2):
                bottom = image.shape[0]
                foundBottom = True
        
        image = image[top:bottom, :]
        cv2.imwrite(f"./output/{path.split('.')[0]}_final_cropped.png", image)

        # LA IMATGE JA ESTÀ RETALLADA I NOMES HI HA LA INFO UTIL
        



    

    # cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    # cnts = sorted(cnts, key=lambda item1: item1[0][0][1])
    # x, y, w, h = cv2.boundingRect(cnts[0])
    # image_cropped = im_gray[y+h:, :]
    # cv2.imwrite(f"./output/{path.split('.')[0]}_cropped.png", image_cropped)  
    
    # cnts = cnts[1:]
    # cnts = sorted(cnts, key=lambda item1: cv2.contourArea(item1, True))
    # cnts = cnts[0:10]
    # i = 0
    # for cnt in cnts:
    #     x, y, w, h = cv2.boundingRect(cnt)
    #     cv2.imwrite(f"./output/{path.split('.')[0]}_{i}.png", image[y:y+h, x:x+w])  
    #     i += 1    



